In [26]:

import pandas as pd
import numpy as np
from datetime import datetime

In [27]:
%matplotlib inline

In [28]:
from math import sin,cos,sqrt,atan2, radians, degrees, atan

In [29]:
input_file = 'raw_data_collected/josh_2018_2019.csv'

In [69]:
raw  =pd.read_csv(input_file, header=None, names=['Date','Time','Lat','Lon','GPS_Alt','Pressure','Pres_Alt','Temp','Batt'])

In [70]:
#remove the first line if its text
raw=raw[1:]
raw.shape

(94702, 9)

In [71]:
raw.Date.unique()

array(['03/15/2018 ', '03/16/2018 ', '60/01/2031 ', '03/17/2018 ',
       '03/17/2019 ', '03/18/2019 ', '03/19/2019 ', '03/20/2019 '],
      dtype=object)

In [72]:
raw = raw[raw.Date=='03/20/2019 ']

In [73]:
raw.shape

(13319, 9)

In [74]:
name = 'josh'
date = datetime(2019,3,20)
output_file = '{}_{}.pkl'.format(name,date.strftime('%d%b%y'))
output_file

'josh_20Mar19.pkl'

In [75]:
raw['Lat_Rad']=raw.Lat.apply(lambda x: radians(x))
raw['Lon_Rad']=raw.Lon.apply(lambda x: radians(x))

In [76]:
raw['Lat_Delta']=raw.Lat[:-1] - raw.Lat[1:].values # current value minus next value
raw['Lon_Delta']=raw.Lon[:-1] - raw.Lon[1:].values
raw['Lat_Rad_Delta']=raw.Lat_Rad[:-1] - raw.Lat_Rad[1:].values
raw['Lon_Rad_Delta']=raw.Lon_Rad[:-1] - raw.Lon_Rad[1:].values

In [77]:
level = 1.0e-06

In [78]:
data = raw[(abs(raw.Lat_Delta) > level) | (abs(raw.Lon_Delta) > level)]

In [79]:
1.0*data.shape[0]/raw.shape[0]

0.8621518131991891

In [80]:
data['GPS_Alt_Delta']=data.GPS_Alt[1:]-data.GPS_Alt[:-1].values #next value minus current

/home/jjc/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
def calc_distance(row):
    
    dlat = row['Lat_Rad_Delta']
    dlon= row['Lon_Rad_Delta']
    lat1 = row['Lat_Rad']
    lat2 = row['Lat_Rad'] + dlat
    #calc haversine distance
    R=6373000.0
    
    a=sin(dlat/2)**2 +cos(lat1) *cos(lat2) *sin(dlon /2)**2
    c= 2 *atan2(sqrt(a),sqrt(1-a))
    return R*c

In [81]:
data['distance']=data.apply(calc_distance,axis=1)

/home/jjc/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
def clip(x):
    
    if x == 0.0 or x is None:
        return 0.00001
    else:
        return x

In [21]:
def calc_gradient(row):
    
    dist= clip(row['distance'])
    height = clip(row['GPS_Alt_Delta'])
    
    return degrees(atan(dist/height ))

In [82]:
#gradient
data['gradient'] = data.apply(calc_gradient,axis=1)

/home/jjc/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
def calc_grad_colour(row):
    
    grad=row['gradient']
    
    if grad >=3.0:
        return 0 #'#FF0000' #red
    elif grad >= -5.0:
        return 1 #'#00FF00' #green
    else:
        return 2#'#0000FF' #blue
        

In [83]:
data['grad_colour']=data.apply(calc_grad_colour,axis=1)

/home/jjc/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
data.to_pickle('processed_data/'+output_file)